In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-235889
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-235889


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "project1ComputeClusterNK"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_config = AmlCompute.provisioning_configuration(
    vm_size = "STANDARD_D2_V2",
    min_nodes=0,
    max_nodes=4,
)

try:
    cluster_handle = ComputeTarget(workspace=ws, name=cluster_name)
    print("Get already existing cluster.")
except:
    print("Could not find such cluster. Create one.")
    cluster_handle = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
cluster_handle.wait_for_completion()


Get already existing cluster.


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(min_value=1e-2, max_value=5e2),
    "--max_iter": choice([25, 50, 75])
})
# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.075,
    evaluation_interval=1, 
    delay_evaluation=5
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=cluster_handle,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=12,
)

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run_with_hyperdrive_opt = exp.submit(config=hyperdrive_config)

In [13]:
RunDetails(run_with_hyperdrive_opt).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
best_result_experiment = run_with_hyperdrive_opt.get_best_run_by_primary_metric()

In [25]:
best_result_experiment.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-06-16-16',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_result_experiment.download_file("outputs/model.pkl", "outputs/hyperdrive_best_model.pkl")
best_result_experiment.register_model(model_name="hyperdrive_opt_best_model", model_path=os.path.join("outputs", "model.pkl"))

In [31]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
DATA_URL =  "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=DATA_URL)

In [32]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [48]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=x.join(y),
    label_column_name="y",
    n_cross_validations=2,
    #compute_target=cluster_handle,
    #max_concurrent_iterations=4,
    enable_early_stopping=True,
    )

In [49]:
# Submit your automl run

run_with_autml_opt = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b6fcc823-3fc7-4460-9f6f-86ae0f700abd,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/06/16 17:56:17 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [ ]:
# Retrieve and save your best automl model.

best_run, best_model = run_with_autml_opt.get_output()
best_run.download_file(os.path.join("outputs", "model.pkl"), os.path.join("outputs", "automl_best_model.pkl"))

In [ ]:
# Delete compute cluster
cluster_handle.delete()